<h1>Algorithms tests</h1>

In [5]:
import sys
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
sys.path.append('/Users/Evgeny/wonderful_programming/fuzzy-fusion-venv/fuzzy-fusion/src/')
from generator.generator import generator
from algorithm.gibbs import gibbs_sampl
from algorithm.gibbs_fuzzy import gibbs_fuzzy
from algorithm.em import em

print 'Python version ' + sys.version
print 'Pandas version ' + pd.__version__

Python version 2.7.10 (default, Oct 23 2015, 18:05:06) 
[GCC 4.2.1 Compatible Apple LLVM 7.0.0 (clang-700.0.59.5)]
Pandas version 0.17.1


In [7]:
s_number = 5
obj_number = 10
cov_list = [0.7]*s_number
p_list = [0.7]*s_number
ground_truth = [random.randint(0, 1) for i in range(obj_number)]
accuracy_list = [0.8]*s_number
# accuracy_list = [0.8, 0.5, 0.6, 0.7, 0.9]
g_true = 0.5

accuracy_for_df = [[i, accuracy_list[i]] for i in range(s_number)]
accuracy_data = pd.DataFrame(accuracy_for_df, columns=['S', 'A'])

em_d = []
g_d = []
gf_d = []
data, g_data = generator(cov_list, p_list, ground_truth, 1, 1.)

em_d.append(em(data=data, accuracy=accuracy_data, truth_obj_list=ground_truth)[0])
for i in range(10):
    while True:
        try:
            g_d.append(gibbs_sampl(data=data, accuracy_data=accuracy_data, truth_obj_list=ground_truth)[0])
            gf_d.append(gibbs_fuzzy(data=data, accuracy_data=accuracy_data, g_data=g_data, truth_obj_list=ground_truth)[0])
            break
        except ZeroDivisionError:
            print 'zero {}'.fprmat(i)

In [8]:
print 'em: {}; {}'.format(np.mean(em_d), np.std(em_d))
print 'g_d: {}; {}'.format(np.mean(g_d), np.std(g_d))
print 'gf_d: {}; {}'.format(np.mean(gf_d), np.std(gf_d))

em: 0.9; 0.0
g_d: 0.854000034188; 0.0407921534431
gf_d: 0.880221421569; 0.0188471399058
